In [1]:
import os
import sys
PATH="/notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations"
sys.path.append( os.path.join( PATH ) )

from allennlp.predictors.predictor import Predictor
import spacy

BERT_PATH=os.path.join(  PATH, "tests/test_files/models", "bert_model", "bert-base-srl-2019.06.17.tar.gz" )
SPACY_PATH=os.path.join( PATH, "tests/test_files/models", "spacy_model", "spacy-textcat" )

#print( f"loading AllenNLP predictor from {BERT_PATH}" )
#bert_model = Predictor.from_path( BERT_PATH, cuda_device=0 )

#print( f"loading spacy model from {SPACY_PATH}" )
#nlp=spacy.load( SPACY_PATH )

#http://docs.allennlp.org/v0.9.0/api/allennlp.predictors.html
from src.reporting_obligations import ReportingObligationsFinder

reporting_obligations_finder = ReportingObligationsFinder(  BERT_PATH, SPACY_PATH, gpu=0  )


loading AllenNLP predictor from /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/models/bert_model/bert-base-srl-2019.06.17.tar.gz
loading spacy model from /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/models/spacy_model/spacy-textcat


In [4]:
#read in cassis 
#read in the json:
import json
import base64
import re
import sys
import os

from typing import Generator, List

from cassis.typesystem import load_typesystem
from cassis.xmi import load_cas_from_xmi
from cassis import Cas

TEMPLATE_PATH=os.path.join( PATH, "tests/test_files/templates/out.html.template" )


with open( os.path.join( PATH , "tests/test_files/typesystems/typesystem.xml" )  , 'rb') as f:
    typesystem = load_typesystem(f)

#with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/double_nested_list_response.json')) as json_file:
#    response = json.load(json_file)

#with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/small_nested_tables_response.json')) as json_file:
#    response = json.load(json_file)
    
#with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/minus_lesser_of_response.json')) as json_file:
#    response = json.load(json_file)  

#with open( os.path.join( PATH, 'tests/test_files/testing_bug/32011D1208-input.json')) as json_file:
#    response = json.load(json_file)  
    
#with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/32002R0063-input.xmi_outputPar_det.json')) as json_file:
#    response = json.load(json_file)    #busy here
    
#with open( os.path.join( PATH, 'tests/test_files/testing_bug/32014R1141-input.xmi_outputPar_det.json')) as json_file:
#    response = json.load(json_file)   #busy here
    
with open( os.path.join( PATH, 'tests/test_files/testing_bug/32019L2121-input.xmi_outputPar_det.json')) as json_file:
    response = json.load(json_file)   #busy here
    
#with open( os.path.join( PATH, 'tests/test_files/testing_bug/32019L2121-input.xmi_outputRO_gpu_2.json')) as json_file:
#    response = json.load(json_file)

#32019L2121-input.xmi_outputRO_gpu_2
    
#with open(  os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/doc_bf4ef384-bd7a-51c8-8f7d-d2f61865d767_response.json')) as json_file:
#    response = json.load(json_file)
    
OUTPUT_PATH=os.path.join( PATH, "tests/test_files/output_reporting_obligations/testing_bug3.html" )
    
decoded_cas=base64.b64decode( response[ 'cas_content' ] ).decode( 'utf-8' )

cas=load_cas_from_xmi( decoded_cas, typesystem=typesystem )

In [5]:
from src.transform import ListTransformer

#from src.transform import get_other_lines, transform_lines, flatten_offsets

#from src.utils import SeekableIterator

OldSofaID = 'html2textView'
NewSofaID = 'ListView'
value_between_tagtype = "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType"
paragraph_type = "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph"

transformer = ListTransformer( cas )
print( "add list view" )
transformer.add_list_view( OldSofaID )


add list view


In [6]:
#test_string=open( "test.txt" ).read()
#test="\n".join([" ".join(test_string.split( "\n" )[0].split(  )[:490])+"|(10,10)" , " ".join(test_string.split( "\n" )[0].split(  )[:20])+"|(10,10)" ])
#transformer.cas.get_view( "ListView" ).sofa_string=test


In [7]:
print( "get reporting obligations" )
import time
start=time.time()
reporting_obligations_finder.process_sentences( cas, ListSofaID='ListView'  )
reporting_obligations_finder.add_xml_to_cas( cas, TEMPLATE_PATH, ROSofaID='ReportingObligationsView' )
reporting_obligations_finder.print_to_html(  TEMPLATE_PATH, OUTPUT_PATH  )
end=time.time()
print( end-start )

get reporting obligations
loading tokenizer en_core_web_sm
ARGM[] ARGM-ADV therefore
ARGM[] ARGM-ADV at least
ARGS[for] ARG2 for members holding shares with voting rights and who voted against the approval of the draft terms
ARGS[to] ARG1 to have the right to exit the company and receive cash compensation for their shares that is equivalent to the value of those shares
ARGS[to] ARG1 to the operation
Writing output to /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/output_reporting_obligations/testing_bug3.html using /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/templates/out.html.template as html template
46.427661418914795


In [10]:
input_sentence="This is a test sentence."
bert_model = Predictor.from_path( BERT_PATH, cuda_device=1 )
parsed_sentence=bert_model.predict(
    sentence=input_sentence
)

In [12]:
parsed_sentence

{'verbs': [{'verb': 'is',
   'description': '[ARG1: This] [V: is] [ARG2: a test sentence] .',
   'tags': ['B-ARG1', 'B-V', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'O']}],
 'words': ['This', 'is', 'a', 'test', 'sentence', '.']}

In [23]:
from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter

tokenizer = SpacyWordSplitter(language="en_core_web_sm", pos_tags=True)
tokenized_sentence=tokenizer.split_words( input_sentence )
bert_model.predict_tokenized( [str( word ) for word in tokenized_sentence] )

{'verbs': [{'verb': 'is',
   'description': '[ARG1: This] [V: is] [ARG2: a test sentence] .',
   'tags': ['B-ARG1', 'B-V', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'O']}],
 'words': ['This', 'is', 'a', 'test', 'sentence', '.']}

In [25]:
len( tokenized_sentence )

6

In [19]:
type( tokenized_sentence[0] )

allennlp.data.tokenizers.token.Token

In [20]:
str( tokenized_sentence[0] )

'This'

In [39]:
from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter

tokenizer = SpacyWordSplitter(language="en_core_web_sm", pos_tags=True) 
len(tokenizer.split_words( test.split( "\n" )[0] ))

532

In [48]:
len( test_string )

2156

In [66]:
if (len( "")  > 2 or len( "test test test test".split()) > 2) :
    print( "sd" ) 

sd


In [51]:
len( test_string.split( ) )

332

In [19]:

len( tokenizer.split_words( test_string ) )

# self._tokenizer = SpacyWordSplitter(language='en_core_web_sm', pos_tags=True) 


2551

In [12]:
import time
start=time.time()
try: 
    parsed_sentence=bert_model.predict(sentence=test[2] )
except RuntimeError:
    print( "RunTimeError, could not parse " )
end=time.time()
print( end-start )

4.5183799266815186


In [13]:
RuntimeError

RuntimeError

In [47]:
def break_long_paragraphs(text: str ) -> list:
    
    if len(text) > 500:
        text = re.sub(r'([^❮❬‖❭❯]{500}(?:[^❮❬‖❭❯.]|[.][^❮❬‖❭❯ ])*[.]) (?=[A-Z])', r'\1\n', text, re.MULTILINE + re.UNICODE)
    
    break_lines = [line.strip() for line in text.split( "\n" ) if line.strip()]

    return break_lines

In [48]:
import re
broken=break_long_paragraphs( test[2] )

In [53]:
broken[0]

"As a key element of possessing European legal status, European political parties and European political foundations should have European legal personality. The acquisition of European legal personality should be subjected to requirements and procedures to protect the interests of the Member State of the seat, of the applicant for European legal status ('the applicant') and of any third parties concerned. In particular, any pre-existing national legal personality should be converted into a European legal personality and any individual rights and obligations that have accrued to the former national legal entity should be transferred to the new European legal entity."

In [46]:
import time
start=time.time()
bert_model.predict( sentence=test[2] )
end=time.time()
print( end-start )

4.548712491989136


In [34]:
test[2][:700]

"As a key element of possessing European legal status, European political parties and European political foundations should have European legal personality. The acquisition of European legal personality should be subjected to requirements and procedures to protect the interests of the Member State of the seat, of the applicant for European legal status ('the applicant') and of any third parties concerned. In particular, any pre-existing national legal personality should be converted into a European legal personality and any individual rights and obligations that have accrued to the former national legal entity should be transferred to the new European legal entity. Moreover, in order to facil"

In [23]:
test[2][:500]

"As a key element of possessing European legal status, European political parties and European political foundations should have European legal personality. The acquisition of European legal personality should be subjected to requirements and procedures to protect the interests of the Member State of the seat, of the applicant for European legal status ('the applicant') and of any third parties concerned. In particular, any pre-existing national legal personality should be converted into a Europe"

In [77]:
test=open( "test.txt" ).read().split( "\n" )
test[1]

'test test and more. And more ❮ And then. And more and then. And more ❯ And more and then. And'

In [78]:
break_long_paragraphs( test[1] )

['test test and more.',
 'And more ❮ And then. And more and then.',
 'And more ❯ And more and then.',
 'And']

In [56]:
test

['test test and more. And more ❮ And then. And more and then. And more❯ And more and then. And']

In [42]:
test

['The termination of European legal personality should be subject to requirements and procedures to protect the interests of the Union, of the Member State of the seat, of the European political party or European political foundation and of any third parties concerned. In particular, if the European political party or European political foundation acquires legal personality under the law of the Member State of its seat, this should be considered as a conversion of the European legal personality and any individual rights and obligations that the former European legal entity has respectively acquired or incurred should be transferred to the national legal entity. Moreover, in order to facilitate continuity of activity, safeguards should be put in place to prevent the Member State concerned from applying prohibitive conditions to such conversion. If the European political party or European political foundation does not acquire legal personality in the Member State of its seat, it should b

In [ ]:
print( "get reporting obligations" )
import time
start=time.time()
reporting_obligations_finder = ReportingObligationsFinder( cas, bert_model, nlp )
reporting_obligations_finder.process_sentences( ListSofaID='ListView'  )
reporting_obligations_finder.add_xml_to_cas( TEMPLATE_PATH, ROSofaID='ReportingObligationsView' )
reporting_obligations_finder.print_to_html(  TEMPLATE_PATH, OUTPUT_PATH  )
end=time.time()
print( end-start )

In [60]:
def break_long_paragraphs(text: str ) -> list:
    
    if len(text) > 10:
        text = re.sub(r'([^❮❬‖❭❯]{10}(?:[^❮❬‖❭❯.]|[.][^❮❬‖❭❯ ])*[.]) (?=[A-Z])', r'\1\n', text, re.MULTILINE + re.UNICODE)
    
    break_lines = [line.strip() for line in text.split( "\n" ) if line.strip()]

    return break_lines

In [64]:
text= "As a key element of possessing European legal status, European political parties and European political foundations should have European legal personality.  ❮ Test and.  ❯ ❯ . The acquisition of European legal personality should be subject to requirements and procedures to protect the interests of the Member State of the seat, of the applicant for European legal status (the applicant) and of any third parties concerned. In particular, any pre-existing national legal personality should be converted into a European legal personality and any individual rights and obligations that have accrued to the former national legal entity should be transferred to the new European legal entity. Moreover, in order to facilitate continuity of activity, safeguards should be put in place to prevent the Member State concerned from applying prohibitive conditions to such conversion. The Member State of the seat should be able to specify which types of national legal persons may be converted into European legal persons, and to withhold its agreement to the acquisition of European legal personality under this Regulation until adequate guarantees are provided, in particular, for the legality of the applicant statutes under the laws of that Member State or for the protection of creditors or holders of other rights in respect of any pre-existing national legal personality."

In [65]:
len( text )

1371

In [66]:
break_long_paragraphs(text)

['As a key element of possessing European legal status, European political parties and European political foundations ❮ should have European legal personality.  ❮ Test and.  ❯ ❯ . The acquisition of European legal personality should be subject to requirements and procedures to protect the interests of the Member State of the seat, of the applicant for European legal status (the applicant) and of any third parties concerned.',
 'In particular, any pre-existing national legal personality should be converted into a European legal personality and any individual rights and obligations that have accrued to the former national legal entity should be transferred to the new European legal entity.',
 'Moreover, in order to facilitate continuity of activity, safeguards should be put in place to prevent the Member State concerned from applying prohibitive conditions to such conversion.',
 'The Member State of the seat should be able to specify which types of national legal persons may be converted

In [30]:
print( "get reporting obligations" )
import time
start=time.time()
reporting_obligations_finder = ReportingObligationsFinder( cas, bert_model, nlp )
reporting_obligations_finder.process_sentences( ListSofaID='ListView'  )
reporting_obligations_finder.add_xml_to_cas( TEMPLATE_PATH, ROSofaID='ReportingObligationsView' )
reporting_obligations_finder.print_to_html(  TEMPLATE_PATH, OUTPUT_PATH  )
end=time.time()
print( end-start )

get reporting obligations
processing sentence: 5. In the case of a sample, only a selection of the credit institutions and other institutions in the potential reporting population shall be asked to report. The variables that shall be estimated by means of the sample are the interest rates and the amounts of new business and the interest rates on outstanding amounts. They are referred to as sampling variables. In order to minimise the risk that the results of a sample survey deviate from the true (unknown) values in the potential reporting population, the sample shall be constructed in such a way that it is representative of the potential reporting population. For the purpose of MFI interest rate statistics a sample shall be considered representative if all the characteristics that are relevant for MFI interest rate statistics and inherent in the potential reporting population are also reflected in the sample. For drawing the initial sample, NCBs may use suitable proxies and models to p

RuntimeError: index out of range: Tried to access index 512 out of table with 511 rows. at /opt/conda/conda-bld/pytorch_1579022030672/work/aten/src/TH/generic/THTensorEvenMoreMath.cpp:418

In [ ]:
3.8721108436584473

In [34]:
len( (test_text*3).split() )

541

In [35]:
test_text

'5. In the case of a sample, only a selection of the credit institutions and other institutions in the potential reporting population shall be asked to report. The variables that shall be estimated by means of the sample are the interest rates and the amounts of new business and the interest rates on outstanding amounts. They are referred to as sampling variables. In order to minimise the risk that the results of a sample survey deviate from the true (unknown) values in the potential reporting population, the sample shall be constructed in such a way that it is representative of the potential reporting population. For the purpose of MFI interest rate statistics a sample shall be considered representative if all the characteristics that are relevant for MFI interest rate statistics and inherent in the potential reporting population are also reflected in the sample. For drawing the initial sample, NCBs may use suitable proxies and models to produce the sampling scheme even if the underly

In [36]:
len(test_text)

1124

In [42]:
text="2.   The data shall contain at least the following elements ❮the identity and contact details of each contribution debtor as determined in accordance with Article 4 of Regulation (EU) No 1163/2014 for the purpose of the supervisory fees ‖ the fee factors determined in accordance with Article 10 of Regulation (EU) No 1163/2014 ‖ whether a contribution debtor is significant in accordance with Article 6(4) of Regulation (EU) No 1024/2013 or is an entity or group in relation to which the ECB has decided in accordance with Article 6(5)(b) of Regulation (EU) No 1024/2013 to exercise directly all of the relevant powers ‖ any data that the ECB has determined in the absence of reporting from a contribution debtor, in accordance with Article 10(5) of Regulation (EU) No 1163/2014 ‖ the validity date underlying the fee calculation of each contribution debtor determining the duration the contribution debtor was subject to the supervisory fee and any change of status in accordance with Article 7(2) of Regulation (EU) No 1163/2014 in the given fee period❯"

text = re.sub(r'([^❮❬‖❭❯]{10}(?:[^❮❬‖❭❯.]|[.][^❮❬‖❭❯ ])*[.]) (?=[A-Z])', r'\1\n', text, re.MULTILINE + re.UNICODE)


In [47]:
text.split( "\n" )

['2.   The data shall contain at least the following elements ❮the identity and contact details of each contribution debtor as determined in accordance with Article 4 of Regulation (EU) No 1163/2014 for the purpose of the supervisory fees ‖ the fee factors determined in accordance with Article 10 of Regulation (EU) No 1163/2014 ‖ whether a contribution debtor is significant in accordance with Article 6(4) of Regulation (EU) No 1024/2013 or is an entity or group in relation to which the ECB has decided in accordance with Article 6(5)(b) of Regulation (EU) No 1024/2013 to exercise directly all of the relevant powers ‖ any data that the ECB has determined in the absence of reporting from a contribution debtor, in accordance with Article 10(5) of Regulation (EU) No 1163/2014 ‖ the validity date underlying the fee calculation of each contribution debtor determining the duration the contribution debtor was subject to the supervisory fee and any change of status in accordance with Article 7(2

In [ ]:
#check performance issue...

In [46]:
'❮'=='❮'

True

In [56]:
def print_break_long_paragraphs(text):
    if len(text) > 10:
        text = re.sub(r'([^❮❬‖❭❯]{10}(?:[^❮❬‖❭❯.]|[.][^❮❬‖❭❯ ])*[.]) (?=[A-Z])', r'\1\n', text, re.MULTILINE + re.UNICODE)
    print(text)

In [57]:
print_break_long_paragraphs( text  )

2.   The data shall contain at least the following elements ❮the identity and contact details of each contribution debtor as determined in accordance with Article 4 of Regulation (EU) No 1163/2014 for the purpose of the supervisory fees ‖ the fee factors determined in accordance with Article 10 of Regulation (EU) No 1163/2014 ‖ whether a contribution debtor is significant in accordance with Article 6(4) of Regulation (EU) No 1024/2013 or is an entity or group in relation to which the ECB has decided in accordance with Article 6(5)(b) of Regulation (EU) No 1024/2013 to exercise directly all of the relevant powers ‖ any data that the ECB has determined in the absence of reporting from a contribution debtor, in accordance with Article 10(5) of Regulation (EU) No 1163/2014 ‖ the validity date underlying the fee calculation of each contribution debtor determining the duration the contribution debtor was subject to the supervisory fee and any change of status in accordance with Article 7(2) 

In [2]:
from src.transform import ListTransformer
from src.reporting_obligations import ReportingObligationsFinder

from src.transform import get_other_lines, transform_lines, flatten_offsets, postprocess_nested_lines

from src.utils import SeekableIterator

#print( f"loading AllenNLP predictor from {BERT_PATH}" )
#bert_model = Predictor.from_path( BERT_PATH )

#print( f"loading spacy model from {SPACY_PATH}" )
#nlp=spacy.load( SPACY_PATH )

OldSofaID = 'html2textView'
NewSofaID = 'ListView'
value_between_tagtype = "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType"
paragraph_type = "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph"

value_between_tagtype_generator=cas.get_view( OldSofaID ).select( value_between_tagtype )        

#value_between_tagtype_generator=get_deepest_child_tags(  cas, OldSofaID , \
#                   value_between_tagtype=value_between_tagtype  )

seek_vbtt=SeekableIterator( iter(value_between_tagtype_generator) )

lines, offsets=get_other_lines( cas , OldSofaID, seek_vbtt, 'root', paragraph_type=paragraph_type )

#flatten_offsets( offsets )

#lines, offsets=postprocess_nested_lines( lines, offsets  )